# Imports

In [1]:
import time

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

torch.__version__
torch.manual_seed(44)

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available, so using", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available, so using CPU only.")

CUDA is available, so using NVIDIA GeForce GTX 1660 Ti


# Data Loading

In [3]:
# now we will take big data as training(main) data on gpu
df = pd.read_csv("Data/Fashion MNIST/fashion-mnist-train.csv")
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Data Preparation

Train Test Split

In [4]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 1:].values, df.iloc[:, 0].values, test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((48000, 784), (12000, 784), (48000,), (12000,))

dataset

In [5]:
class CustomDataset(Dataset):

    def __init__(self, features, labels):
        features = features / 255.0  # Normalize pixel values to [0, 1]
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
    
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)

dataloader

In [6]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True)

# Model Training

defining model

In [7]:
class ANNModel(nn.Module):

    def __init__(self, num_features):
        super().__init__()

        self.network = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),

            nn.Linear(128, 64),
            nn.ReLU(),

            nn.Linear(64, 10), # here dont need softmax as CrossEntropyLoss does it internally
        )

    def forward(self, x):
        out = self.network(x)

        return out
    
model = ANNModel(num_features=x_train.shape[1])
model = model.to(device) # move model to device

defining loss and optimizer

In [8]:
criterion = nn.CrossEntropyLoss()

epochs = 100
learning_rate = 0.1
optimizer = optim.SGD(params=model.parameters(), lr=learning_rate)

training pipeline

In [9]:
model.train() # set model to training mode

for epoch in range(epochs):

    loss = 0.0
    for batch in train_loader:
        batch_x_train, batch_y_train = batch
        batch_x_train, batch_y_train = batch_x_train.to(device), batch_y_train.to(device)
        
        # forward pass
        batch_y_pred = model(batch_x_train)

        # calculate loss
        batch_loss = criterion(batch_y_pred, batch_y_train)

        # backward pass
        optimizer.zero_grad()
        batch_loss.backward()

        # update parameters
        optimizer.step()

        loss += batch_loss.item()

    avg_loss = loss / len(train_loader)
    print(f"Epoch: {epoch + 1}, Loss: {avg_loss:.4f}")

Epoch: 1, Loss: 0.7475
Epoch: 2, Loss: 0.4656
Epoch: 3, Loss: 0.4120
Epoch: 4, Loss: 0.3819
Epoch: 5, Loss: 0.3597
Epoch: 6, Loss: 0.3420
Epoch: 7, Loss: 0.3259
Epoch: 8, Loss: 0.3149
Epoch: 9, Loss: 0.3048
Epoch: 10, Loss: 0.2961
Epoch: 11, Loss: 0.2858
Epoch: 12, Loss: 0.2752
Epoch: 13, Loss: 0.2723
Epoch: 14, Loss: 0.2631
Epoch: 15, Loss: 0.2542
Epoch: 16, Loss: 0.2492
Epoch: 17, Loss: 0.2422
Epoch: 18, Loss: 0.2367
Epoch: 19, Loss: 0.2309
Epoch: 20, Loss: 0.2256
Epoch: 21, Loss: 0.2224
Epoch: 22, Loss: 0.2155
Epoch: 23, Loss: 0.2109
Epoch: 24, Loss: 0.2068
Epoch: 25, Loss: 0.2027
Epoch: 26, Loss: 0.1967
Epoch: 27, Loss: 0.1930
Epoch: 28, Loss: 0.1897
Epoch: 29, Loss: 0.1869
Epoch: 30, Loss: 0.1825
Epoch: 31, Loss: 0.1793
Epoch: 32, Loss: 0.1760
Epoch: 33, Loss: 0.1706
Epoch: 34, Loss: 0.1686
Epoch: 35, Loss: 0.1665
Epoch: 36, Loss: 0.1617
Epoch: 37, Loss: 0.1590
Epoch: 38, Loss: 0.1559
Epoch: 39, Loss: 0.1544
Epoch: 40, Loss: 0.1501
Epoch: 41, Loss: 0.1455
Epoch: 42, Loss: 0.1462
E

# Model Evaluating

training accuracy

In [11]:
model.eval()  # set model to evaluation mode

accuracy = []

with torch.no_grad():
    for batch in train_loader:
        batch_x_train, batch_y_train = batch
        batch_x_train, batch_y_train = batch_x_train.to(device), batch_y_train.to(device)

        outputs = model(batch_x_train)
        _, batch_y_pred = torch.max(outputs, dim=1)
        batch_accuracy = (batch_y_pred == batch_y_train).float().mean().item()
        accuracy.append(batch_accuracy)

overall_accuracy = np.mean(accuracy)
print(f"Training Accuracy: {overall_accuracy * 100:.2f}%")

Training Accuracy: 98.29%


testing accuracy

In [12]:
model.eval()  # set model to evaluation mode

accuracy = []

with torch.no_grad():
    
    for batch in test_loader:
        batch_x_test, batch_y_test = batch
        batch_x_test, batch_y_test = batch_x_test.to(device), batch_y_test.to(device)

        outputs = model(batch_x_test)
        _, batch_y_pred = torch.max(outputs, dim=1)
        batch_accuracy = (batch_y_pred == batch_y_test).float().mean().item()
        accuracy.append(batch_accuracy)

overall_accuracy = np.mean(accuracy)
print(f"Testing Accuracy: {overall_accuracy * 100:.2f}%")

Testing Accuracy: 88.94%
